In [1]:
import sys

sys.path.append("..")

In [2]:
from aurelio_sdk import AsyncAurelioClient
import os

client = AsyncAurelioClient(api_key=os.environ["AURELIO_API_KEY"], base_url="http://localhost:8000", debug=True)

# Extract text from PDFs and video MP4 files and urls

### From file

In [10]:
import magic
from aurelio_sdk import ExtractResponse
import aiofiles


# file_path = "data/pdf/adaptive_semantic_search.pdf"
# file_path = "data/video/how_to_overcome_our_mistakes.mp4"
file_path = "data/pdf/test"

import tempfile
import os

with tempfile.NamedTemporaryFile(delete=False, suffix='.md') as temp_file:
    temp_file_path = temp_file.name

async with aiofiles.open(file_path, "rb") as f:
    file_content = await f.read()

async with aiofiles.open(temp_file_path, "wb") as temp_f:
    await temp_f.write(file_content)

async with aiofiles.open(temp_file_path, "rb") as f:
    temp_file_content = await f.read()

content_type = magic.from_file(temp_file_path, mime=True)

if content_type is None:
    content_type = "application/octet-stream"

print(content_type)


response_pdf_file: ExtractResponse = await client.extract_file(
    file=temp_file_content, quality="low", chunk=True, wait=-1, enable_polling=True
)

os.unlink(temp_file_path)

response_pdf_file

inode/x-empty


APIError: API request failed with status 400: {'detail': 'Unsupported file type: `application/x-empty`, available types: application/pdf, video/mp4'}.

In [ ]:
from aurelio_sdk import ExtractResponse

# From a local file
file_path = "data/video/how_to_overcome_our_mistakes.mp4"

async with aiofiles.open(file_path, "rb") as f:
    file_content = await f.read()

response_video_file: ExtractResponse = await client.extract_file(
    file=file_content, quality="low", chunk=True, wait=-1, enable_polling=True
)

response_video_file

### From URL

In [ ]:
from aurelio_sdk import ExtractResponse

# From URL
url = "https://arxiv.org/pdf/2408.15291"
response_pdf_url: ExtractResponse = await client.extract_url(
    url=url, quality="low", chunk=True, wait=-1, enable_polling=True
)

response_pdf_url

In [ ]:
from aurelio_sdk import ExtractResponse

# From URL
url = "https://storage.googleapis.com/gtv-videos-bucket/sample/ForBiggerMeltdowns.mp4"
response_video_url: ExtractResponse = await client.extract_url(
    url=url, quality="low", chunk=True, wait=-1, enable_polling=True
)

response_video_url

# Get document status and handle timeouts

In [ ]:
from aurelio_sdk import ExtractResponse

# From a local file
file_path = "data/pdf/adaptive_semantic_search.pdf"

async with aiofiles.open(file_path, "rb") as f:
    file_content = await f.read()

response_pdf_file: ExtractResponse = await client.extract_file(
    file=file_content, quality="high", chunk=True, wait=10, enable_polling=True
)

# Get document status and document id
print("Status:", response_pdf_file.status)
print("Document ID:", response_pdf_file.document.id)

In [ ]:
# Get document status and response
document_response: ExtractResponse = await client.get_document(
    document_id=response_pdf_file.document.id, timeout=1
)

print("Status:", document_response.status)

In [ ]:
# Use a pre-built function, which helps to avoid long hanging requests (Recommended)
document_response = await client.wait_for(
    document_id=response_pdf_file.document.id, wait=300
)

In [ ]:
document_response